In [10]:
import itertools
from surprise import accuracy
from collections import defaultdict

In [1]:
class RecommenderMetrics:
    
    def MAE(predictions):
        return accuracy.mae(predictions,verbose=False)
    
    def RMSE(predictions):
        return accuracy.rmse(predictions,verbose=False)
    
    def GetTopN(predictions,n = 10,minimumRating = 4.0):
        topN = defaultdict(list)
        
        for userID,movieID,actualRating,estimatedRating,_ in predictions:
            if (estimatedRating >= minimumRating):
                topN[int(userID)].append((int(movieID),estimatedRating))
        
        for userID,ratings in topN.items():
            ratings.sort(key=lambda x:x[1], reverse = True)
            topN[int(userID)] = ratings[:n]
        return topN
    
    # To See how often we recommended a movie the user actually rated
    def HitRate(topNPredicted,leftOutPredictions):
        hits = 0
        total = 0
        
        for leftOut in leftOutPredictions:
            userID = leftOut[0]
            leftOutMovieID = leftOut[1]
            hit = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if(int(leftOutMovieID) == int(movieID)):
                    hit = True
                    break
            if(hit):
                hits += 1
            total += 1
            
        return hits/total
    # To See how often we recommended a movie the user actually liked
    def CumulativeHitRate(topNPredicted,leftOutPredictions,ratingCutOff = 0.0):
        hits = 0
        total = 0
        
        for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
            
            if(actualRating >= ratingCutOff):
                hit = False
                for movieID,predictedRating in topNPredicted[int(userID)]:
                    if(int(leftOutMovieID) == int(movieID)):
                        hit = True
                        break
                if(hit):
                    hits += 1
                total += 1
                
        return hits/total
    
    # It just break down hit rate by rating value
    def RatingHitRate(topNPredicted,leftOutPredictions):
        hits = defaultdict(float)
        total = defaultdict(float)
        
        for userID,leftOutMovieID,actualRating,estimatedRating,_ in leftOutPredictions:
            hit = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutMovieID) == movieID):
                    hit = True
                    break
            if(hit):
                hits[actualRating] += 1
            
            total[actualRating] += 1
            
        for rating in sorted(hits.keys()):
            print(rating,hits[rating]/total[rating])
            
    def AverageReciprocalHitRank(topNPredicted,leftOutPredictions):
        totalSum = 0
        total = 0
        
        for userID,leftOutMovieID,actualRating,estimatedRating,_ in leftOutPredictions:
            hitRank = 0
            rank = 0 
            for movieID, predictedRating in topNPredicted[int(userID)]:
                rank = rank + 1
                if (int(leftOutMovieID) == movieID):
                    hitRank = rank
                    break
                    
            if(hitRank > 0):
                totalSum += 1.0/hitRank
            
            total += 1
            
        return totalSum/total
    
    # What percentage of users have at least one good recommendation
    def UserCoverage(topNPredicted,numUsers,ratingThreshold = 0):
        hits = 0 
        for userID in topNPredicted.keys():
            hit  = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if(predictedRating >= ratingThreshold):
                    hit = True
                    break
            if(hit):
                hits += 1
                
        return hits/numUsers
    
    def Diversity(topNPredicted,simsAlgo):
        n = 0
        total = 0
        simsMatrix = simsAlgo.compute_similarities()
        for userID in topNPredicted.keys():
            pairs = itertools.combinations(topNPredicted[userID],2)
            for pair in pairs:
                movie1 = pair[0][0]
                movie2 = pair[1][0]
                innerID1 = simsAlgo.trainset.to_inner_iid(str(movie1))
                innerID2 = simsAlgo.trainset.to_inner_iid(str(movie2))
                similarity = simsMatrix[innerID1][innerID2]
                total += similarity
                n += 1
                
        S = total/n
        return (1-S)

    
    def Novelty(topNPredicted,rakings):
        n = 0
        total = 0
        for userID in topNPredicted.keys():
            for rating in topNPredicted[userID]:
                movieID = rating[0]
                rank = rakings[movieID]
                total += rank
                n += 1
        return total/n       